In [9]:
import pandas as pd
import numpy as np
import os
import scipy.stats as st
from collections import Counter
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from random import choices

In [2]:
path = '../trendanalysis_finegrain/'
path_miss = '../trendanalysis_finegrain/missing_data/'
path_fulltext = '../trendanalysis_finegrain/fulltext/'
files = [i for i in os.listdir(path) if 'edited' not in i and '.csv' in i]
analysis_predictions_df = pd.DataFrame()
for file in files:
    analysis_predictions_df = pd.concat([analysis_predictions_df, pd.read_csv(os.path.join(path, file), header=0)])
    analysis_predictions_df = pd.concat([analysis_predictions_df, pd.read_csv(os.path.join(path_miss, file), header=0).drop(['IF'], axis=1)])
    analysis_predictions_df = pd.concat([analysis_predictions_df, pd.read_csv(os.path.join(path_fulltext, file), header=0).drop(['IF'], axis=1)])
    

/home/lanj3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
/home/lanj3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':
/home/lanj3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and sil

In [3]:
analysis_predictions_df = analysis_predictions_df[~analysis_predictions_df['CONSORT_item'].isin(['3b', '6b'])]
analysis_predictions_df.shape

(2647035, 10)

In [5]:
Counter(analysis_predictions_df.disciplines)

Counter({nan: 90645,
         'other': 796245,
         'immunology': 75525,
         'gastroenterology_hepatology': 83145,
         'urology_nephrology': 68250,
         'infectious': 79905,
         'general': 278070,
         'gynacology_reproduction': 108405,
         'heart_vascular': 208440,
         'anesthesiology': 153435,
         'psychiatry': 143190,
         'oncology': 144285,
         'pediatrics': 65400,
         'surgery': 134700,
         'neurology': 95730,
         'endocrinology_metabolism': 121665})

In [6]:
analysis_predictions_df['disciplines'] = analysis_predictions_df['disciplines'].fillna('other')

In [12]:
list_discipline = {}
for displine, df_displine in analysis_predictions_df.groupby('disciplines'):
    list_discipline[displine] = df_displine

In [19]:
record = []
for i in list_discipline:
    for j in list_discipline:
        if list(list_discipline.keys()).index(i)<list(list_discipline.keys()).index(j):
            record.append([i, j, st.ttest_ind(list_discipline[i], list_discipline[j])[1]])

In [22]:
pd.DataFrame(record).to_csv('displine.csv', index=False)

In [17]:
output = []
label_figure = []
numbers = []
error = []
list_discipline = {}
for displine, df_displine in analysis_predictions_df.groupby('disciplines'):
    # print(stats.ttest_ind(arr1, arr2))
    print(displine)
    report_ratio = []
    for pmid, df_displine_pmid in df_displine.groupby('PMID'):
        report_ratio.append(list(df_displine_pmid.is_reported).count('Y')/15)
    list_discipline[displine]=report_ratio
    print(np.mean(report_ratio))
    numbers.append(np.mean(report_ratio)*100)
    sampling = []
    for i in range(1000):
        sampling.append(sum(choices(report_ratio, k=len(report_ratio)))/len(report_ratio))
#     sorted(sampling)
    cil, ciu=np.quantile(sampling, 0.025), np.quantile(sampling, 0.975)
#     cil, ciu = st.t.interval(confidence=0.95, df=len(report_ratio)-1, loc=np.mean(report_ratio), scale=st.sem(report_ratio))
    print(cil, ciu)
    print(len(report_ratio))
    print(np.mean(report_ratio)-cil)
    output.append([displine, np.mean(report_ratio), cil, ciu, ciu-np.mean(report_ratio), np.mean(report_ratio)-cil])
    error.append((np.mean(report_ratio)-cil)*100)
    print('------------------------')

anesthesiology
0.4277837520774269
0.4247459836412936 0.4306515788444734
10229
0.0030377684361332924
------------------------
endocrinology_metabolism
0.4167015986520363
0.4131903176755995 0.4204582254551598
8111
0.003511280976436826
------------------------
gastroenterology_hepatology
0.4593421131757772
0.4539055264898683 0.464634975043601
5543
0.005436586685908906
------------------------
general
0.4632826266767361
0.4602291689142919 0.46621713956916316
18538
0.0030534577624442005
------------------------
gynacology_reproduction
0.48103869747705363
0.4767111756837806 0.4855041280383772
7227
0.004327521793273059
------------------------
heart_vascular
0.38587603147188637
0.38310053252733056 0.3885306323162388
13896
0.00277549894455581
------------------------
immunology
0.42810989738497185
0.42306388613042434 0.4326004634227111
5035
0.005046011254547511
------------------------
infectious
0.46607846818096493
0.4608097115324503 0.4708597709780397
5327
0.0052687566485146164
-------------

In [18]:
pd.DataFrame(output)

,0,1,2,3,4,5
0,anesthesiology,0.427784,0.424746,0.430652,0.002868,0.003038
1,endocrinology_metabolism,0.416702,0.413190,0.420458,0.003757,0.003511
2,gastroenterology_hepatology,0.459342,0.453906,0.464635,0.005293,0.005437
3,general,0.463283,0.460229,0.466217,0.002935,0.003053
4,gynacology_reproduction,0.481039,0.476711,0.485504,0.004465,0.004328
5,heart_vascular,0.385876,0.383101,0.388531,0.002655,0.002775
6,immunology,0.428110,0.423064,0.432600,0.004491,0.005046
7,infectious,0.466078,0.460810,0.470860,0.004781,0.005269
8,neurology,0.460744,0.456250,0.465540,0.004796,0.004493
9,oncology,0.438909,0.435601,0.442500,0.003591,0.003308
